### Connect to Graph

In [ ]:
Disconnect-MgGraph

### Local environment file example

clientconfig.json

```json
{
    "TenantId": "guid",
    "ClientId": "guid",
    "Thumbprint": "3457982afshjdk2384723",
}
```

In [ ]:
$config = Get-Content C:\temp\clientconfiguration.json | ConvertFrom-Json
Connect-MgGraph -ClientId $config.ClientID -TenantId $config.TenantId -CertificateThumbprint $config.Thumbprint

In [ ]:
$FormatEnumerationLimit=-1
Get-MgContext

### Function

In [ ]:
function Invoke-GraphHunt {
    param(
        [Parameter(Mandatory, Position=0)]
        [string]$query
    )

    $uri  = "https://graph.microsoft.com/v1.0/security/runHuntingQuery"
    $body = @{ Query = $query } | ConvertTo-Json

    $resp = Invoke-MgGraphRequest -Method POST -Uri $uri -Body $body -ContentType "application/json" -OutputType PSObject

    $results = foreach ($row in $resp.Results) {
        $flat = [ordered]@{}
        foreach ($p in $row.PSObject.Properties) {
            if ($p.Name -like '*@odata.type') { continue }
            $v = $p.Value
            if ($v -is [System.Array]) {
                $flat[$p.Name] = ($v | ForEach-Object {
                    if ($_ -is [pscustomobject]) { $_ | ConvertTo-Json -Compress } else { $_ }
                }) -join '; '
            } elseif ($v -is [pscustomobject]) {
                $flat[$p.Name] = $v | ConvertTo-Json -Compress
            } else {
                $flat[$p.Name] = $v
            }
        }
        [pscustomobject]$flat
    }

    $logDir = ".\log"
    if (-not (Test-Path $logDir)) { New-Item -ItemType Directory -Path $logDir | Out-Null }

    $csvPath = Join-Path $logDir ("KQLResults_{0}.csv" -f (Get-Date -Format 'yyyyMMdd_HHmmss'))
    # $results | Export-Csv -Path $csvPath -NoTypeInformation -Encoding UTF8
    # Write-Host "Results exported to $csvPath"

    $timestamp = Get-Date -Format 'yyyyMMdd_HHmmss'
    $path = Join-Path $logDir "KQLResults_$timestamp.tabulator.html"

    $view = $results

    $json = $view | ConvertTo-Json -Depth 10

    $jsonSafe = $json -replace '</script>', '<\/script>'

    $first = $view | Select-Object -First 1
    $colNames = @()
    if ($first) { $colNames = $first.PSObject.Properties.Name }

    $cols = foreach ($c in $colNames) {
        $isWide = $c -match 'Subject|Body|Url|Query|Evidence|Summary'
        $widthGrow = if ($isWide) { 2 } else { 1 }
@"
{ title: "$c", field: "$c", headerFilter: "input", sorter: "string", resizable: true, widthGrow: $widthGrow }
"@
    }
    $colsJs = $cols -join ",`n"

    $html = @"
<!doctype html>
<html>
<head>
  <meta charset="utf-8" />
  <title>KQL – Results</title>
  <link href="file:///C:/vs/scripts/assets/tabulator/tabulator.min.css" rel="stylesheet">
  <style>
    body { font-family: Segoe UI, Arial; margin: 14px; }
    .header { display:flex; align-items:baseline; gap:12px; margin-bottom:10px; }
    h2 { margin:0; font-weight:600; }
    .meta { color:#666; font-size:12px; }
    .controls { display:flex; gap:10px; align-items:center; margin: 10px 0 12px; }
    input[type="text"] { padding:6px 8px; width: 360px; }
    button { padding:6px 10px; cursor:pointer; }
    #grid { border:1px solid #ddd; }
    #err { color:#b00020; white-space:pre-wrap; font-family: Consolas, monospace; margin-top:10px; }
  </style>
</head>
<body>
  <div class="header">
    <h2>KQL Results</h2>
    <div class="meta">Generated: $([DateTime]::Now.ToString("yyyy-MM-dd HH:mm:ss"))</div>
  </div>

  <div class="controls">
    <input id="globalSearch" type="text" placeholder="Global search (all columns)..." />
    <button id="clearFilters">Clear filters</button>
    <button id="downloadCsv">Download CSV</button>
  </div>

  <div id="grid"></div>
  <div id="err"></div>

  <!-- Data payload -->
  <script id="data-json" type="application/json">
$jsonSafe
  </script>

  <script src="file:///C:/vs/scripts/assets/tabulator/tabulator.min.js"></script>
  <script>
    function showErr(e){
      const el = document.getElementById("err");
      el.textContent = (e && e.stack) ? e.stack : String(e);
    }

    try {
      const raw = document.getElementById("data-json").textContent;
      const data = JSON.parse(raw);

      const table = new Tabulator("#grid", {
        data: data,
        layout: "fitDataStretch",
        height: "80vh",
        movableColumns: true,
        resizableColumns: true,
        pagination: "local",
        paginationSize: 50,
        paginationSizeSelector: [25, 50, 100, 250, 500],
        columns: [
          $colsJs
        ],
      });

      // Global search across all columns
      document.getElementById("globalSearch").addEventListener("input", (e) => {
        const val = (e.target.value || "").trim().toLowerCase();
        if (!val) { table.clearFilter(true); return; }
        table.setFilter((rowData) => Object.values(rowData).some(v =>
          (v ?? "").toString().toLowerCase().includes(val)
        ));
      });

      document.getElementById("clearFilters").addEventListener("click", () => {
        document.getElementById("globalSearch").value = "";
        table.clearFilter(true);
        table.clearHeaderFilter();
        table.clearSort();
      });

      document.getElementById("downloadCsv").addEventListener("click", () => {
        table.download("csv", "Results_$timestamp.csv");
      });

    } catch (e) {
      showErr(e);
    }
  </script>
</body>
</html>
"@

    $html | Set-Content -Path $path -Encoding UTF8
    Start-Process $path
    "Opened: $path"
}

### Queries

- KQL query needs to be inside the powershell double quoted here-string

Example:
```powershell
$query = 
@"
EmailEvents
| where Timestamp >= ago(24h)
| take 200
"@

Invoke-GraphHunt $query
```

#### Ready to copy template

In [ ]:
$query = 
@"
asdfasdfasdfasdfasdfasdfasdf
"@

Invoke-GraphHunt $query

#### Query List

In [ ]:
$query = 
@"
EmailEvents
| where Timestamp >= ago(24h)
| take 100
"@

Invoke-GraphHunt $query

In [ ]:
$query = 
@"
CloudAppEvents
| where Timestamp >= ago(30d)
| where ActionType == "SoftDelete"
| extend UserId = tostring(RawEventData.UserId)
| summarize DeleteCount = count() by UserId
| top 5 by DeleteCount desc
"@

Invoke-GraphHunt $query

# Custom Detection Rules Demo

#### Send message with url in attachment

In [ ]:
Disconnect-MgGraph

In [ ]:
$config = Get-Content C:\temp\clientconfiguration.json | ConvertFrom-Json
Connect-MgGraph -ClientId $config.ClientID -TenantId $config.TenantId -CertificateThumbprint $config.Thumbprint

In [ ]:
# Config
$FromUser = "user@fabrikam.com"
$ToUser = "user@contoso.com"
$FilePath = "C:\temp\outer.zip"

# Read & encode attachment

$FileBytes = [System.IO.File]::ReadAllBytes($FilePath)
$Base64File = [Convert]::ToBase64String($FileBytes)
$FileName = [System.IO.Path]::GetFileName($FilePath)

# Build message

$Message = @{
    Subject      = "Test email with attachment (Graph SDK) 1"
    Body         = @{
        ContentType = "Text"
        Content     = "This email was sent using Microsoft Graph PowerShell SDK."
    }
    ToRecipients = @(
        @{
            EmailAddress = @{
                Address = $ToUser
            }
        }
    )
    Attachments  = @(
        @{
            "@odata.type" = "#microsoft.graph.fileAttachment"
            Name          = $FileName
            ContentType   = "application/octet-stream"
            ContentBytes  = $Base64File
        }
    )
}

# Send mail
Send-MgUserMail `
    -UserId $FromUser `
    -Message $Message `
    -SaveToSentItems

### KQL samples

#### Detect url in attachments

In [ ]:
let badDomains = dynamic([
    "adobe.com",
    "spamlink.contoso.com",
    "toyota.com"
]);
let urlsInAttachments =
    EmailUrlInfo
    | where Timestamp > ago(1h)
    | where UrlLocation == "Attachment"
    | where UrlDomain in (badDomains)
    | project
        UrlTimestamp = Timestamp,
        NetworkMessageId,
        Url,
        UrlDomain,
        UrlLocation;
let zipAttachments =
    EmailAttachmentInfo
    | where Timestamp > ago(1h)
    | where FileType =~ "zip" or FileExtension =~ "zip"
    | project
        NetworkMessageId,
        FileName,
        FileType,
        FileExtension;
urlsInAttachments
| join kind=inner zipAttachments on NetworkMessageId
| join kind=inner (
    EmailEvents
    | where Timestamp > ago(1h)
    | where EmailDirection == "Inbound"
    | project
        Timestamp,
        ReportId,
        NetworkMessageId,
        InternetMessageId,
        RecipientEmailAddress,
        SenderFromAddress,
        SenderFromDomain,
        Subject
) on NetworkMessageId
| project
    Timestamp,
    ReportId,
    NetworkMessageId,
    InternetMessageId,
    RecipientEmailAddress,
    SenderFromAddress,
    SenderFromDomain,
    Subject,
    FileName,
    FileExtension,
    Url,
    UrlDomain,
    UrlLocation

#### External recipient rate limit

In [ ]:
EmailEvents
| where Timestamp > ago(24h)
    and EmailDirection == "Outbound"
| extend RecipientType = iff(isnotempty(DistributionList), "DistributionList", "IndividualRecipient")
| summarize 
    RecipientCount = countif(RecipientType == "IndividualRecipient") 
    + dcountif(RecipientType, RecipientType == "DistributionList"),
    Timestamp = max(Timestamp),
    ReportId = tostring(max(ReportId))
    by SenderFromAddress
| where RecipientCount > 2000
| project 
    Timestamp, 
    ReportId, 
    SenderFromAddress, 
    RecipientCount
| order by RecipientCount desc

#### Recipient rate limit

In [ ]:
EmailEvents
| where Timestamp > ago(24h)
| where EmailDirection != "Inbound"
| where isnotempty(SenderFromAddress)
| where isnotempty(RecipientEmailAddress) or isnotempty(DistributionList)
| extend RecipientKey =
    iff(
    isnotempty(DistributionList),
    strcat("DL:", DistributionList),
    strcat("RCPT:", RecipientEmailAddress)
)
| summarize
    RecipientCount = dcount(RecipientKey),
    arg_max(Timestamp, ReportId)
    by SenderFromAddress, InternetMessageId
| summarize
    RRLCount = sum(RecipientCount),
    arg_max(Timestamp, ReportId)
    by SenderFromAddress
| where RRLCount >= 10000
| project 
    Timestamp, 
    ReportId, 
    SenderFromAddress, 
    RRLCount

In [ ]:
Disconnect-MgGraph

### Email Remediations via Graph

In [ ]:
function Invoke-EmailRemediation {
    param (
        [Parameter(Mandatory)]
        [string]$NetworkMessageId,

        [Parameter(Mandatory)]
        [string]$RecipientEmailAddress,

        [Parameter(Mandatory)]
        [ValidateSet("moveToJunk", "moveToDeletedItems", "softDelete", "hardDelete")]
        [string]$Action
    )

    $config = Get-Content "C:\temp\clientconfiguration.json" | ConvertFrom-Json
    Connect-MgGraph -ClientId $config.ClientId -TenantId $config.TenantId -CertificateThumbprint $config.Thumbprint -NoWelcome

    Write-Host "NetworkMessageId: $NetworkMessageId"
    Write-Host "RecipientEmailAddress: $RecipientEmailAddress"
    Write-Host "Action: $Action"

    $payload = @{
        displayName          = "Email Action from Graph SDK"
        description          = "Delete or move email"
        severity             = "medium"
        action               = $Action
        remediateSendersCopy = $true
        analyzedEmails       = @(
            @{
                networkMessageId      = $NetworkMessageId
                recipientEmailAddress = $RecipientEmailAddress
            }
        )
    }

    $jsonBody = $payload | ConvertTo-Json -Depth 10

    Write-Host "Request Payload:`n$jsonBody"

    try {
        $response = Invoke-MgGraphRequest `
            -Uri "https://graph.microsoft.com/beta/security/collaboration/analyzedEmails/remediate" `
            -Method POST `
            -Body $jsonBody `
            -ContentType "application/json"

        Write-Host "`nSuccess: The request has been accepted."
        return $response
    }
    catch {
        Write-Host "Error: $($_.Exception.Message)"
        if ($null -ne $_.Exception.Response) {
            Write-Host "HTTP Status: $($_.Exception.Response.StatusCode)"
        }
    }
}

In [ ]:
EmailEvents
| where Timestamp > ago (10d)
| where EmailDirection != "Outbound"
| project NetworkMessageId, RecipientEmailAddress, Subject
| sample 25

In [ ]:
# Remediate a specific email
Invoke-EmailRemediation `
    -NetworkMessageId "ad396742-6d44-450a-08e8-08de5b688129" `
    -RecipientEmailAddress "user@contoso.com" `
    -Action "hardDelete"

In [ ]:
# Remediate multiple emails from a list
$emailsToRemediate = @(
    @{ NetworkMessageId = "abc123"; Recipient = "alice@contoso.com" },
    @{ NetworkMessageId = "def456"; Recipient = "bob@contoso.com" },
    @{ NetworkMessageId = "ghi789"; Recipient = "carol@contoso.com" }
)

foreach ($email in $emailsToRemediate) {
    Invoke-EmailRemediation `
        -NetworkMessageId $email.NetworkMessageId `
        -RecipientEmailAddress $email.Recipient `
        -Action "moveToDeletedItems"
}

### Powershell UI

In [ ]:
Add-Type -AssemblyName PresentationFramework

# Load config and connect to Graph once at startup
$config = Get-Content "C:\temp\clientconfiguration.json" | ConvertFrom-Json
Connect-MgGraph -ClientId $config.ClientId -TenantId $config.TenantId -CertificateThumbprint $config.Thumbprint -NoWelcome

# Function to remediate an email using Graph SDK
function Remediate-Email {
    param (
        [string]$NetworkMessageId,
        [string]$RecipientEmailAddress,
        [string]$Action
    )

    $payload = @{
        displayName          = "Email Action from Graph SDK"
        description          = "Delete or move email"
        severity             = "medium"
        action               = $Action
        remediateSendersCopy = $true
        analyzedEmails       = @(@{
                networkMessageId      = $NetworkMessageId
                recipientEmailAddress = $RecipientEmailAddress
            })
    }

    $jsonBody = $payload | ConvertTo-Json -Depth 10

    Write-Host "Payload:" $jsonBody

    try {
        $response = Invoke-MgGraphRequest `
            -Uri "https://graph.microsoft.com/beta/security/collaboration/analyzedEmails/remediate" `
            -Method POST `
            -Body $jsonBody `
            -ContentType "application/json"

        if ($null -ne $response -and $response -ne "") {
            Write-Host "Response:" ($response | ConvertTo-Json -Depth 10)
            [System.Windows.MessageBox]::Show("Email action successfully processed.", "Success")
        }
        else {
            Write-Host "No response body received, but the action was successful."
            [System.Windows.MessageBox]::Show("Email action successfully processed.", "Success")
        }
    }
    catch {
        Write-Host "Error Message:" $_.Exception.Message
        if ($_.Exception.Response) {
            $errorResponse = $_.Exception.Response.GetResponseStream() | New-Object System.IO.StreamReader
            $responseBody = $errorResponse.ReadToEnd()
            Write-Host "HTTP Response Body:" $responseBody
        }
        [System.Windows.MessageBox]::Show("Error: $($_.Exception.Message)", "Error")
    }
}

# Create GUI window
function Show-Gui {
    $Window = New-Object System.Windows.Window
    $Window.Title = "Graph SDK Email Remediation"
    $Window.SizeToContent = "WidthAndHeight"
    $Window.WindowStartupLocation = "CenterScreen"

    $Grid = New-Object System.Windows.Controls.Grid
    $Grid.Margin = "10"
    $Window.Content = $Grid

    for ($i = 0; $i -lt 4; $i++) {
        $Row = New-Object System.Windows.Controls.RowDefinition
        $Grid.RowDefinitions.Add($Row)
    }
    for ($i = 0; $i -lt 2; $i++) {
        $Col = New-Object System.Windows.Controls.ColumnDefinition
        $Grid.ColumnDefinitions.Add($Col)
    }

    $Grid.ColumnDefinitions[0].Width = "Auto"
    $Grid.ColumnDefinitions[1].Width = "*"

    $Controls = @{ }
    $Fields = @(
        @{ Label = "Network Message ID:"; Variable = "NetworkMessageId" },
        @{ Label = "Recipient Email Address:"; Variable = "RecipientEmailAddress" },
        @{ Label = "Action:"; Variable = "Action"; Options = @("softDelete", "hardDelete", "moveToJunk", "moveToDeletedItems") }
    )

    for ($i = 0; $i -lt $Fields.Count; $i++) {
        $Label = New-Object System.Windows.Controls.Label
        $Label.Content = $Fields[$i].Label
        $Label.Margin = "5"
        $Label.HorizontalAlignment = "Left"
        $Grid.Children.Add($Label)
        [System.Windows.Controls.Grid]::SetRow($Label, $i)
        [System.Windows.Controls.Grid]::SetColumn($Label, 0)

        if ($Fields[$i].ContainsKey("Options")) {
            $ComboBox = New-Object System.Windows.Controls.ComboBox
            $ComboBox.Margin = "5"
            $ComboBox.Width = 300
            foreach ($Option in $Fields[$i].Options) {
                $ComboBox.Items.Add($Option)
            }
            $ComboBox.SelectedIndex = 0
            $Grid.Children.Add($ComboBox)
            [System.Windows.Controls.Grid]::SetRow($ComboBox, $i)
            [System.Windows.Controls.Grid]::SetColumn($ComboBox, 1)
            $Controls[$Fields[$i].Variable] = $ComboBox
        }
        else {
            $TextBox = New-Object System.Windows.Controls.TextBox
            $TextBox.Margin = "5"
            $TextBox.Width = 300
            $Grid.Children.Add($TextBox)
            [System.Windows.Controls.Grid]::SetRow($TextBox, $i)
            [System.Windows.Controls.Grid]::SetColumn($TextBox, 1)
            $Controls[$Fields[$i].Variable] = $TextBox
        }
    }

    $Button = New-Object System.Windows.Controls.Button
    $Button.Content = "Submit"
    $Button.Margin = "10"
    $Button.HorizontalAlignment = "Center"
    $Button.Add_Click({
            $Inputs = @{ }
            foreach ($Field in $Fields) {
                $Control = $Controls[$Field.Variable]
                if ($Control -is [System.Windows.Controls.ComboBox]) {
                    $Inputs[$Field.Variable] = $Control.SelectedItem
                }
                else {
                    $Inputs[$Field.Variable] = $Control.Text
                }
            }

            if ($Inputs.Values -contains "") {
                [System.Windows.MessageBox]::Show("All fields are required.", "Validation Error")
                return
            }

            Remediate-Email `
                -NetworkMessageId $Inputs.NetworkMessageId `
                -RecipientEmailAddress $Inputs.RecipientEmailAddress `
                -Action $Inputs.Action
        })
    $Grid.Children.Add($Button)
    [System.Windows.Controls.Grid]::SetRow($Button, $Fields.Count)
    [System.Windows.Controls.Grid]::SetColumnSpan($Button, 2)

    $Window.ShowDialog()
}

Show-Gui

### Web App
### Teams Bot
### Logic Apps/Power Automate
### Sentinel
### Power BI